In [79]:
# IMPORTANTO BIBLIOTECAS A SEREM TRABALHADAS

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from plyer import notification 
import requests
from tqdm import tqdm

In [80]:
def alerta(nivel, base, etapa, erro=""):
    '''
        Alerta de falha de carregamento de base de dados
    '''
    now = str(datetime.now())

    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

In [95]:
def tabelas_bd():
    '''
        Retorna um dataframe com as tabelas do banco de dados.
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema
def salva_bd(df, nome_tabela):
    '''
        Salva dataframe df na tabela nome_tabela.
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True
def carrega_bd(nome_tabela):
    '''
        Carrega tabela nome_tabela num dataframe. 
    '''
    conn = sqlite3.connect('coderhouse.db')

    # Executar uma consulta SELECT na tabela 'produtos' e converter em um DataFrame
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)

    conn.close()

    return df  

In [82]:
tabelas_bd()

,name


In [83]:
def get_json_api(url):
    '''
        Request GET url e retorna json de saida
    '''

    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        return data_json
    
    # Erro
    raise Exception(f"erro request,  {response.status_code} - {url}\n{response.text}")

In [84]:
def get_base_pokemons_url():
    '''
        request completo iterativo da tabela pokemons_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/pokemon"
    df_pokemons_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_pokemons = pd.DataFrame(json_data['results'])

        #append/concat  
        df_pokemons_full = pd.concat([df_pokemons_full,df_pokemons],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_pokemons_full

In [85]:
def get_base_habilidades_url():
    '''
        request completo iterativo da tabela habilidades_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/ability"
    df_habilidades_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_habilidade = pd.DataFrame(json_data['results'])

        #append/concat  
        df_habilidades_full = pd.concat([df_habilidades_full,df_habilidade],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_habilidades_full

In [86]:
def get_base_especies_url():
    '''
        request completo iterativo da tabela especie_url
        retorna dataframe da tabela 
    '''
    next_url = "https://pokeapi.co/api/v2/pokemon-species"
    df_especie_full = pd.DataFrame()
    while next_url is not None:
        print(next_url, end='\r')
        
        # GET 
        json_data = get_json_api(next_url)

        #para dataframe
        df_especie = pd.DataFrame(json_data['results'])

        #append/concat  
        df_especie_full = pd.concat([df_especie_full,df_especie],ignore_index=True)

        #next page
        next_url = json_data['next']
    
    return df_especie_full

In [87]:
def get_base_pokemons():

    df_pokemons_url = carrega_bd("pokemons_url")

    df_pokemons_full = pd.DataFrame()

    for url in tqdm(df_pokemons_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id              = json_data['id']
        name            = json_data['name']
        abilities       = json_data['abilities']
        height          = json_data['height']
        weight          = json_data['weight']
        base_experience = json_data['base_experience']

        sr_pokemon = pd.Series({
            "url": url,
            "pokemon_id": id,
            "name": name,
            "height": height,
            "weight": weight,
            "base_experience": base_experience,
            "habilidades": str(abilities)
        })

        #append/concat 
        df_pokemons_full = pd.concat([df_pokemons_full,sr_pokemon.to_frame().T],ignore_index=True)
        
    return df_pokemons_full

In [88]:
def get_base_habilidades():

    df_habilidades_url = carrega_bd("habilidades_url")

    df_habilidades_full = pd.DataFrame()

    for url in tqdm(df_habilidades_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        generation     = json_data['generation']['name']
        is_main_series = json_data['is_main_series']
        effect         = "\n---\n".join([effect['effect'] for effect in json_data['effect_entries']])

        sr_habilidade = pd.Series({
            "url": url,
            "ability_id":id,
            "name": name,
            "generation": generation,
            "is_main_series": is_main_series,
            "effect": effect
        })

        #append/concat 
        df_habilidades_full = pd.concat([df_habilidades_full,sr_habilidade.to_frame().T],ignore_index=True)

    return df_habilidades_full

In [89]:
def get_base_especies():

    df_especie_url = carrega_bd("especie_url")

    df_especie_full = pd.DataFrame()

    for url in tqdm(df_especie_url['url'].values):

        #GET
        json_data = get_json_api(url)

        #para dataframe e append
        id             = json_data['id']
        name           = json_data['name']
        color          = json_data['color']['name']
        growth_rate    = json_data['growth_rate']['name']        
        

        sr_especie = pd.Series({
            "url": url,
            "specie_id":id,
            "name": name,
            "color": color,
            "growth_rate": growth_rate        
        })

        #append/concat 
        df_especie_full = pd.concat([df_especie_full,sr_especie.to_frame().T],ignore_index=True)

    return df_especie_full

In [90]:
def etapa_extracao():
    
    #################################################
    print("  > Base pokemons_url:\n")
    try:
        df_pokemons_url = get_base_pokemons_url()
        salva_bd(df_pokemons_url,"pokemons_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "pokemons_url", 
               etapa = "EXTRACAO", 
               erro=e)
    
    #################################################
    print("  > Base habilidades_url:\n")
    try:
        df_habilidades_url = get_base_habilidades_url()
        salva_bd(df_habilidades_url,"habilidades_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "habilidades_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base especie_url:\n")
    try:
        df_especie_url = get_base_especies_url()
        salva_bd(df_especie_url,"especie_url")
    except Exception as e:
        alerta(nivel = 3, 
               base = "especie_url", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base pokemons:\n")
    try:
        df_pokemons_full = get_base_pokemons()
        salva_bd(df_pokemons_full,"pokemons")
    except Exception as e:
        alerta(nivel = 3, 
               base = "pokemons", 
               etapa = "EXTRACAO", 
               erro=e)

    
    #################################################
    print("  > Base habilidades:\n")
    try:
        df_habilidades_full = get_base_habilidades()
        salva_bd(df_habilidades_full,"habilidades")
    except Exception as e:
        alerta(nivel = 3, 
               base = "habilidades", 
               etapa = "EXTRACAO", 
               erro=e)
        
    #################################################
    print("  > Base especies:\n")
    try:
        df_especie_full = get_base_especies()
        salva_bd(df_especie_full,"especies")
    except Exception as e:
        alerta(nivel = 3, 
               base = "especies", 
               etapa = "EXTRACAO", 
               erro=e)

In [91]:
etapa_extracao()

  > Base pokemons_url:

  > Base habilidades_url:/pokemon?offset=1300&limit=20

  > Base especie_url:i/v2/ability?offset=360&limit=70

  > Base pokemons:/api/v2/pokemon-species?offset=1020&limit=50



100%|██████████| 1302/1302 [09:31<00:00,  2.28it/s]


  > Base habilidades:



100%|██████████| 367/367 [02:40<00:00,  2.29it/s]


  > Base especies:



100%|██████████| 1025/1025 [07:34<00:00,  2.25it/s]


In [96]:
tabelas_bd()

,name
0,pokemons_url
1,habilidades_url
2,especie_url
3,pokemons
4,habilidades
5,especies


In [97]:
carrega_bd('especies')

,url,specie_id,name,color,growth_rate
0,https://pokeapi.co/api/v2/pokemon-species/1/,1,bulbasaur,green,medium-slow
1,https://pokeapi.co/api/v2/pokemon-species/2/,2,ivysaur,green,medium-slow
2,https://pokeapi.co/api/v2/pokemon-species/3/,3,venusaur,green,medium-slow
3,https://pokeapi.co/api/v2/pokemon-species/4/,4,charmander,red,medium-slow
4,https://pokeapi.co/api/v2/pokemon-species/5/,5,charmeleon,red,medium-slow
...,...,...,...,...,...
1020,https://pokeapi.co/api/v2/pokemon-species/1021/,1021,raging-bolt,yellow,slow
1021,https://pokeapi.co/api/v2/pokemon-species/1022/,1022,iron-boulder,gray,slow
1022,https://pokeapi.co/api/v2/pokemon-species/1023/,1023,iron-crown,blue,slow
1023,https://pokeapi.co/api/v2/pokemon-species/1024/,1024,terapagos,blue,slow
